# Transfer Learning with Tensorflow Part 2: Fine-Tuning

In previous notebook, we learned about transfer learning feature extraction, now it's time to learn about a new kind of transfer learning: `fine-tuning`.

## Creating helper functions 

In previous notebooks, we've created a bunch of helper functions, now we could rewrite them all, however this is tedious.

So it's a good idea to put functions you'll want to use again in a script you can download and import into your notebooks (or elsewhere).

In [1]:
import wget 

In [2]:
wget.download('https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py')

100% [..............................................................................] 10246 / 10246

'helper_functions.py'

In [3]:
# Import helper functions we're going to use in this notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

## Let's get some data

This time we're going to see how we can use pretrained models within `tf.keras.applications` and apply them to our own problem (recognizing images of food).


In [4]:
# Check out how many images and subdirectories are in our dataset 
walk_through_dir('10_food_classes_10_percent')

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

In [5]:
# Create training and test directory paths 
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'


In [8]:
import tensorflow as tf 

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    image_size=IMG_SIZE,
    label_mode='categorical',
    batch_size=BATCH_SIZE
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir,
    image_size=IMG_SIZE,
    label_mode='categorical',
    batch_size=BATCH_SIZE
)


Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [9]:
train_data_10_percent

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [10]:
# Check out the class names of our dataset 
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [11]:
# see an example of a batch of data 
for images, labels in train_data_10_percent.take(1): # unravel a batch
     print(images, labels)
    

tf.Tensor(
[[[[2.23229599e+02 1.86229599e+02 8.22295914e+01]
   [2.27816330e+02 1.90816330e+02 8.68163300e+01]
   [2.31607132e+02 1.93607132e+02 9.26071396e+01]
   ...
   [6.21326485e+01 2.38417950e+01 5.98976803e+00]
   [6.32602386e+01 2.16173820e+01 4.85206604e+00]
   [6.12141113e+01 1.95712547e+01 2.98453188e+00]]

  [[2.20500000e+02 1.83525497e+02 7.73775558e+01]
   [2.20214294e+02 1.83214294e+02 7.89336777e+01]
   [2.21770416e+02 1.83770416e+02 8.33112259e+01]
   ...
   [6.63571014e+01 2.94132233e+01 3.19893765e+00]
   [6.80765457e+01 2.81377678e+01 2.78571439e+00]
   [6.53365631e+01 2.34079933e+01 5.96906424e-01]]

  [[2.15132660e+02 1.78994904e+02 6.78367386e+01]
   [2.13642868e+02 1.76642868e+02 7.05714264e+01]
   [2.16091843e+02 1.77923462e+02 7.95969391e+01]
   ...
   [7.24439163e+01 3.48265762e+01 4.59137261e-02]
   [7.98724670e+01 4.06734467e+01 3.87243795e+00]
   [7.55713196e+01 3.37856064e+01 0.00000000e+00]]

  ...

  [[2.47362259e+02 2.40362259e+02 1.86010223e+02]
   [2